In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize

# Função para carregar o modelo GloVe pré-treinado
def load_glove_model(glove_file):
    print("Carregando modelo GloVe...")
    glove_model = {}
    with open(glove_file, 'r', encoding="utf-8") as f:
        for line in f:
            # Tentar carregar a linha, ignorar se houver erro de conversão
            try:
                # Substituir vírgulas por pontos e remover possíveis caracteres indesejados
                split_line = line.replace(',', '.').split()
                
                # A primeira parte da linha deve ser a palavra
                word = split_line[0]
                
                # O restante da linha deve ser o vetor numérico
                vector = np.array(split_line[1:], dtype=float)
                
                # Adicionar ao dicionário
                glove_model[word] = vector
            
            except ValueError:
                # Ignorar linhas que não possam ser convertidas para float
                print(f"Erro ao carregar linha: {line.strip()} - Linha ignorada.")
                continue
    
    print(f"Modelo GloVe carregado com {len(glove_model)} palavras.")
    return glove_model

# Especifique o caminho para o arquivo GloVe
glove_file = '../Models/glove_s50.txt'  # Altere para o caminho correto

# Carregar o modelo GloVe
glove_model = load_glove_model(glove_file)

# Verificar o vetor de uma palavra (por exemplo, "computer")
word = "computer"
if word in glove_model:
    print(f"Vetor para a palavra '{word}':\n", glove_model[word])
else:
    print(f"A palavra '{word}' não está no vocabulário.")


Carregando modelo GloVe...
Erro ao carregar linha: 00 % 0.238321 0.428413 -0.052815 0.285182 -0.132642 0.074867 -0.156668 -0.136985 -0.121049 -0.014800 -0.075628 0.235240 -0.303932 -0.146515 -0.018260 -0.028394 -0.083194 0.216203 -0.331248 -0.320542 0.290555 0.066519 0.027326 0.177007 -0.077733 0.084656 -0.085467 -0.072151 0.092679 -0.172570 0.111656 0.072783 0.106959 0.265846 -0.015970 0.132251 -0.033092 0.087407 0.063407 0.112884 -0.029037 0.012749 0.086987 0.042678 0.080592 -0.028452 0.153930 -0.126905 -0.129614 0.174028 - Linha ignorada.
Erro ao carregar linha: 三藏法師玄奘奉　詔譯 0.126251 0.044369 0.216259 0.243329 -0.122597 -0.006277 -0.092619 0.039339 0.070852 -0.266326 -0.191690 0.025615 0.212125 -0.113988 0.213476 0.223095 -0.217400 0.038813 0.008619 -0.062391 -0.040933 -0.204107 0.026779 0.220275 0.225373 0.140602 -0.049127 0.076664 0.141255 -0.213412 0.094084 -0.171865 -0.038348 -0.050799 0.028407 0.193167 0.067945 0.151280 0.089024 -0.142448 -0.097696 -0.004128 0.130937 0.088114 0.0

In [19]:
import pandas as pd
# Exemplo de dataset com títulos de artigos de pesquisa científica
data = {
    'document': [
        "Deep learning for image recognition and classification",
        "Quantum computing applications in cryptography",
        "Advances in renewable energy systems and technologies",
        "Blockchain technology and its impact on financial systems",
        "Natural language processing techniques for text classification",
        "Artificial intelligence in healthcare and medical diagnosis",
        "Data mining techniques for big data analytics",
        "Cybersecurity challenges in cloud computing environments",
        "Machine learning algorithms for predictive modeling",
        "Internet of things and smart cities"
    ]
}

df = pd.DataFrame(data)

# Tokenizar os documentos
df['tokens'] = df['document'].apply(lambda x: word_tokenize(x.lower()))

# Função para converter um documento ou consulta em um vetor GloVe médio
def get_average_vector(tokens_list, glove_model):
    vectors = [glove_model[word] for word in tokens_list if word in glove_model]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(100)  # Usar um vetor de zeros se nenhuma palavra estiver no modelo

# Criar vetores médios para os documentos
df['vector'] = df['tokens'].apply(lambda x: get_average_vector(x, glove_model))

# Função para pesquisar documentos com base na similaridade semântica
def pesquisar_documentos(consulta, df, glove_model, top_n=3):
    # Pré-processar a consulta e convertê-la em vetor GloVe
    consulta_tokens = word_tokenize(consulta.lower())
    consulta_vector = get_average_vector(consulta_tokens, glove_model)
    
    # Calcular a similaridade de cosseno entre a consulta e os documentos
    df['similarity'] = df['vector'].apply(lambda x: cosine_similarity([consulta_vector], [x])[0][0])
    
    # Ordenar os documentos pela similaridade e retornar os top_n mais relevantes
    documentos_relevantes = df.sort_values(by='similarity', ascending=False).head(top_n)
    
    return documentos_relevantes[['document', 'similarity']]

# Exemplo de uso: consulta do usuário


Documentos mais relevantes para a consulta: 'Applications of machine learning in healthcare':

                                            document  similarity
1     Quantum computing applications in cryptography    0.945626
5  Artificial intelligence in healthcare and medi...    0.937943
0  Deep learning for image recognition and classi...    0.934165


In [20]:
consulta = "Applications of machine learning in healthcare"
documentos_relevantes = pesquisar_documentos(consulta, df, glove_model, top_n=3)

# Exibir os resultados da pesquisa
print(f"Documentos mais relevantes para a consulta: '{consulta}':\n")
print(documentos_relevantes)


Documentos mais relevantes para a consulta: 'Applications of machine learning in healthcare':

                                            document  similarity
1     Quantum computing applications in cryptography    0.945626
5  Artificial intelligence in healthcare and medi...    0.937943
0  Deep learning for image recognition and classi...    0.934165
